<a href="https://colab.research.google.com/github/bnanik/Shared_Task_SemEval2023/blob/main/Prompt_based_classifier_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  !pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install openprompt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 17.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import torch
import transformers
import datasets
from transformers import AutoModelForSequenceClassification

In [ ]:
df_train = pd.read_csv('/content/NewCombined_train_file.csv')
df_train["text"] = df_train["text"].astype(str)
# df_train['label']=df_train['label_sexist'].fillna(df_train['sexist'])
# df_train.drop(columns=['label_sexist', 'sexist'], inplace=True)
# df_train.rename(columns={"label": "sexist"}, inplace=True)
df_train.head()

,text,sexist
0,MENTION3481 i didn't even know random was an o...,not sexist
1,Bottom two should've gone! #mkr,not sexist
2,MENTION3111 MENTION3424 ladyboner deserves so ...,not sexist
3,She shall now be known as Sourpuss #MKR #KatAn...,not sexist
4,Tarah W threw a bunch of women under the bus s...,not sexist


In [ ]:
df_validation = pd.read_csv('/content/NewTest_set_EDOS.csv')
df_validation["text"] = df_validation["text"].astype(str)
df_validation.rename(columns={"label_sexist": "sexist"}, inplace=True)
df_validation.head()

,text,sexist
0,Gynocentrism/feminism go hand in hand but this...,not sexist
1,Let her choke on your cock and then hard next her,sexist
2,"If you're not banging her brains out, it ain't...",not sexist
3,"So pussy, does this mean that you represent no...",sexist
4,You’re the one who’s sounding like an incel he...,not sexist


In [ ]:
df_dev = pd.read_csv('/content/dev_task_a_entries.csv')
df_dev['labels'] = 0 
df_dev.shape

(2000, 3)

In [ ]:
df_dev.head()

,rewire_id,text,labels
0,sexism2022_english-11999,I do not try to hard. I do what guys who get w...,0
1,sexism2022_english-9895,That sucks. We elected a Navy Seal in Missouri...,0
2,sexism2022_english-11623,Walk away and never look back. If she reaches ...,0
3,sexism2022_english-19035,basically I placed more value on her than I sh...,0
4,sexism2022_english-895,"NYT and Asian ""part ways."" But why on earth wa...",0


In [ ]:
df_test_codalab = pd.read_csv('/content/test_task_a_entries.csv')
df_test_codalab['labels'] = 0 
df_test_codalab.shape

(4000, 3)

In [ ]:
df_test_codalab.head()

,rewire_id,text,labels
0,sexism2022_english-7207,Here on the Oregon coast it doesn't snow. It's...,0
1,sexism2022_english-10731,"""That tall man must certaily be better with wo...",0
2,sexism2022_english-11374,"hit the rode chode, this ain't the subreddit f...",0
3,sexism2022_english-7356,The lawyer chick will shoot down the McDonald'...,0
4,sexism2022_english-11976,"True, it's totally hating on females to want t...",0


In [ ]:
# data = pd.read_csv('/content/train_all_tasks.csv')
# df = data.filter(['text', 'label_sexist'])
# # df.columns = ['text', 'label']
# df["text"] = df["text"].astype(str)
# df.head(2)

### Encoding labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train['labels'] = le.fit_transform(df_train['sexist'])
df_validation['labels'] = le.fit_transform(df_validation['sexist'])
# df.rename(columns={'comment_plain':'text', 'act': 'labels'}, inplace=True)

In [ ]:
df_validation.drop(['sexist'], axis=1, inplace=True)

### Split data

In [ ]:
from sklearn.model_selection import train_test_split
train_ratio = 0.90
validation_ratio = 0.10
# test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
training_data, validation_data = train_test_split(df_train, stratify=df_train['labels'], test_size=1 - train_ratio, random_state=25)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
# _ , testing_data = train_test_split(df_test, stratify=df_test['labels'], test_size=0.99, random_state=25) 
# _, dev_data = train_test_split(df_dev, test_size=2000, random_state=25)


In [ ]:
from sklearn.model_selection import train_test_split
train_ratio = 0.80
validation_ratio = 0.10
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
training_data, remaining = train_test_split(df, stratify=df['labels'], test_size=1 - train_ratio, random_state=25)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
validation_data, testing_data, = train_test_split(remaining, stratify=remaining['labels'], test_size=test_ratio/(test_ratio + validation_ratio), random_state=25) 


In [ ]:
from datasets import Dataset, DatasetDict
raw_dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "validation": Dataset.from_pandas(df_validation),
    "dev": Dataset.from_pandas(df_dev),
    "test": Dataset.from_pandas(df_test_codalab)
})
    

raw_dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'sexist', 'labels'],
        num_rows: 24831
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 2800
    })
    dev: Dataset({
        features: ['rewire_id', 'text', 'labels'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['rewire_id', 'text', 'labels'],
        num_rows: 4000
    })
})

In [ ]:
from openprompt.data_utils import InputExample

dataset = {}
for split in ['train', 'validation', 'dev', 'test']:
    dataset[split] = []
    for data in raw_dataset[split]:
        input_example = InputExample(text_a = data['text'], label=int(data['labels'])) #, guid=data['__index_level_0__']
        dataset[split].append(input_example)
print(dataset['train'][0])


/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


{
  "guid": null,
  "label": 0,
  "meta": {},
  "text_a": "MENTION3481 i didn't even know random was an option!",
  "text_b": "",
  "tgt_text": null
}



### Load the PLM related things provided by openprompt

In [ ]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-large")

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


### Constructing Template

In [ ]:
from openprompt.prompts import ManualTemplate
template_text = '{"placeholder":"text_a"}. This text is {"mask"}.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

To better understand how does the template wrap the example, we visualize one instance.


In [ ]:
wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': "MENTION3481 i didn't even know random was an option!", 'loss_ids': 0, 'shortenable_ids': 1}, {'text': '. This text is', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': '.', 'loss_ids': 0, 'shortenable_ids': 0}], {'label': 0}]


In [ ]:
# Now, the wrapped example is ready to be pass into the tokenizer, hence producing the input for language models.
# You can use the tokenizer to tokenize the input by yourself, but we recommend using our wrapped tokenizer, which is a wrapped tokenizer tailed for InputExample.
# The wrapper has been given if you use our `load_plm` function, otherwise, you should choose the suitable wrapper based on
# the configuration in `openprompt.plms.__init__.py`.
# Note that when t5 is used for classification, we only need to pass <pad> <extra_id_0> <eos> to decoder.
# The loss is calcaluted at <extra_id_0>. Thus passing decoder_max_length=3 saves the space

wrapped_t5tokenizer = WrapperClass(max_seq_length=512, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

In [ ]:
# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))

{'input_ids': [3, 11810, 9215, 3710, 4959, 3, 23, 737, 31, 17, 237, 214, 6504, 47, 46, 1182, 55, 3, 5, 100, 1499, 19, 32099, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Let's convert the whole dataset into the input format

In [ ]:
model_inputs = {}
for split in ['train', 'validation', 'dev', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

### Define a DataLoader

In [ ]:
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 24831it [00:27, 915.63it/s]


In [ ]:
le.classes_

array(['not sexist', 'sexist'], dtype=object)

## Define the verbalizer

In [ ]:

# In classification, we need to define the verbalizer, which is a mapping from logits on the vocabulary to the final label probability. Let's have a look at the verbalizer details:

from openprompt.prompts import ManualVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
myverbalizer = ManualVerbalizer(tokenizer, num_classes=2,
                        label_words=[["not sexist"], ["sexist"]])

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do


Parameter containing:
tensor([[[   59,     3,     7, 12135]],

        [[    3,     7, 12135,     0]]])
tensor([[-0.8555, -0.5535],
        [-0.1008, -2.3450]])


## Train

In [ ]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

for epoch in range(4):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step %100 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0, average loss: 0.4797193333506584
Epoch 0, average loss: 0.514820929855227
Epoch 0, average loss: 0.4648399877939419
Epoch 0, average loss: 0.4246383774690238
Epoch 0, average loss: 0.40671451922433116
Epoch 0, average loss: 0.38879981582503825
Epoch 0, average loss: 0.3728153139799122
Epoch 0, average loss: 0.36015842089653705
Epoch 0, average loss: 0.3548937062927385
Epoch 0, average loss: 0.35016891921218896
Epoch 0, average loss: 0.34447340681863586
Epoch 0, average loss: 0.3411839382375207
Epoch 0, average loss: 0.3377342774571863
Epoch 0, average loss: 0.3336005342452602
Epoch 0, average loss: 0.3298590801077989
Epoch 0, average loss: 0.32494018908937927
Epoch 0, average loss: 0.32355067912810664
Epoch 0, average loss: 0.31876564065290486
Epoch 0, average loss: 0.3165567386354042
Epoch 0, average loss: 0.313963165813489
Epoch 0, average loss: 0.313731771515246
Epoch 0, average loss: 0.3113602897280572
Epoch 0, average loss: 0.3079020377251909
Epoch 0, average loss: 0.3064

In [ ]:
validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
print(acc)

tokenizing: 2800it [00:03, 896.45it/s]


In [ ]:
len(allpreds)

2800

In [ ]:
from sklearn import metrics

print("Accuracy:", metrics.accuracy_score(alllabels, allpreds))
print("F1:",metrics.f1_score(alllabels, allpreds, average="macro"))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(alllabels, allpreds, average="macro"))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(alllabels, allpreds, average="macro"))

In [ ]:
dev_dataloader = PromptDataLoader(dataset=dataset["dev"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

dev_allpreds = []
dev_alllabels = []
for step, inputs in enumerate(dev_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    dev_alllabels.extend(labels.cpu().tolist())
    dev_allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

acc = sum([int(i==j) for i,j in zip(dev_allpreds, dev_alllabels)])/len(dev_allpreds)
print(acc)

In [ ]:
df_dev['label_pred'] = le.inverse_transform(dev_allpreds)
df_dev.drop(columns=['labels', 'text'],axis=1, inplace=True)
df_dev.to_csv('/content/dev_task_a_labeled_t5_large.csv', index=False)

In [ ]:
test_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=512, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

test_allpreds = []
test_alllabels = []
for step, inputs in enumerate(test_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    test_alllabels.extend(labels.cpu().tolist())
    test_allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

acc = sum([int(i==j) for i,j in zip(test_allpreds, test_alllabels)])/len(test_allpreds)
print(acc)

tokenizing: 2000it [00:02, 946.84it/s]


0.8365


In [ ]:
# from sklearn import metrics

# print("Accuracy:", metrics.accuracy_score(alllabels, allpreds))
# print("F1:",metrics.f1_score(alllabels, allpreds, average="macro"))

# # Model Precision: what percentage of positive tuples are labeled as such?
# print("Precision:",metrics.precision_score(alllabels, allpreds, average="macro"))

# # Model Recall: what percentage of positive tuples are labelled as such?
# print("Recall:",metrics.recall_score(alllabels, allpreds, average="macro"))

In [ ]:
len(test_allpreds)

2000

In [ ]:
df_test_codalab.tail()

,rewire_id,text,labels
1995,sexism2022_english-13861,"None ,but some of the girls I bang are on ecst...",0
1996,sexism2022_english-6864,"It's a constant isnt it, whenever women fall b...",0
1997,sexism2022_english-17934,yep. No fapping means you have to chase the va...,0
1998,sexism2022_english-4138,I didn't say you had to stay with a crazy bitc...,0
1999,sexism2022_english-18351,Just get a good looking hooker you weak minded...,0


In [ ]:
df_test_codalab['label_pred'] = le.inverse_transform(test_allpreds)
df_test_codalab.drop(columns=['labels', 'text'],axis=1, inplace=True)
df_test_codalab.to_csv('/content/test_task_a_labeled_t5_large.csv', index=False)